<a href="https://colab.research.google.com/github/smishr97/38616-Neural-Networks-Deep-Learning/blob/main/ShivamMishra_NN_reference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 36-616: Neural Network and Deep Learning
### Homework #1: Neural Network from ***PyTorch***
**Name: Shivam Mishra** <br>
**AndrewID: shivammi**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from typing import Optional, List, Tuple, Dict
import plotly.express as px
import plotly.graph_objs as go
plt.style.use('bmh')


## For CUDA based devices
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
    
# ## For MacOS based devices
# if torch.backends.mps.is_available():
#     device = torch.device("mps")
# else:
#     device = torch.device("cpu")

## Define the layers for the NN

In [ ]:
class SingleLayerMLP(nn.Module):
    """constructing a neural network with Pytorch"""

    def __init__(self, indim, outdim, hidden_dim=1):
        """
        Constructor for the SingleLayerMLP class.

        Args:
        - indim (int): The input dimension of the network.
        - outdim (int): The output dimension of the network.
        - hidden_dim (int): The number of hidden units in the network. Default is 1.
        """

        super(SingleLayerMLP, self).__init__()
        self.indim = indim
        self.outdim = outdim
        self.hidden_dim = hidden_dim

        # Define the layers of the network
        self.linear1 = nn.Linear(self.indim, self.hidden_dim)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(self.hidden_dim, self.hidden_dim)
        # self.relu2 = nn.ReLU()
        # self.linear3 = nn.Linear(self.hidden_dim, self.outdim)

    def forward(self, x):
        """
        Performs a forward pass of the network.

        Args:
        - x (torch.Tensor): Input tensor of shape (batch_size, input_dim)

        Returns:
        - out (torch.Tensor): Output tensor of shape (batch_size, output_dim)
        """

        out = self.linear1(x)  # Apply the first linear transformation
        out = self.relu(out)  # Apply the nonlinearity
        out = self.linear2(out)  # Apply the second linear transformation
        # out = self.relu2(out)  # Apply the nonlinearity
        # out = self.linear3(out)  # Apply the final linear transformation
        return out  # Return the output tensor

## Transform the data

In [ ]:
class DS(Dataset):
    """
    A PyTorch dataset class for loading data in batches.

    X: Input tensor of shape (num_samples, num_features)
    Y: Target tensor of shape (num_samples,)
    """

    def __init__(self, X: np.ndarray, Y: np.ndarray):
        self.length = len(X)  # Total number of samples in the dataset
        self.X = X  # Input tensor
        self.Y = Y  # Target tensor

    def __getitem__(self, idx):
        """
        Returns a single sample from the dataset.

        idx: Index of the sample to retrieve

        Returns:
        A tuple containing the input tensor and target value for the sample at the given index
        """
        x = self.X[idx, :]  # Get the input tensor for the given index
        y = self.Y[idx]  # Get the target value for the given index
        return (x, y)

    def __len__(self):
        """
        Returns the total number of samples in the dataset.
        """
        return self.length

## Train and Test function

In [ ]:
# Function to train and test a model
def train_test(train_loader, test_loader, model, num_epochs, optimizer, loss_fn):
    
    # Initialize lists to store loss and accuracy values
    training_loss = []
    testing_loss = []
    training_accuracy = []
    test_accuracy = []
    train_acc = []
    
    # Loop through epochs
    for epoch in range(num_epochs):
        running_loss = 0.0
        train_correct = 0.0
        total_samples = 0
        
        # Loop through batches in training dataset
        for batch_idx, (data, target) in enumerate(train_loader):
            data = data.to(torch.float32)   # Transpose input and change the datatype to keep consistent 
            # with the default pytorch weight matrix data type
            target = target.to(torch.long)   # Transpose labels 
            # labels_onehot = torch.from_numpy(labels2onehot(target))   # Convert labels to one-hot encoding
            
            # Forward Pass 
            output = model.forward(data)   # Pass input through the model to get output
            loss = loss_fn(output, target)   # Calculate loss using cross-entropy

            # Backward Pass
            optimizer.zero_grad() # sets optimizer to zero grad to remove previous epoch gradients
            loss.backward() # Backpropagate the gradients through the model
            optimizer.step() # Update the model parameters
            
            running_loss += loss.item()* data.size(0)  # Add loss to running loss
            _, predicted = torch.max(output, dim=1)  # get predicted class
            train_correct += (predicted == target).sum().item()  # update total correct

            total_samples += data.size(0)  # update total samples
        
        # Append average training loss and accuracy to lists
        training_loss.append(running_loss / total_samples)
        training_accuracy.append(train_correct / total_samples)
        
        # Evaluate model on test dataset
        test_loss = 0.0
        test_correct = 0.0
        total_samples = 0
        with torch.no_grad():
            for data, target in test_loader:
                data = data.to(torch.float32)
                target = target.to(torch.long)
                # target = torch.from_numpy(labels2onehot(target))

                # Forward Pass
                output = model.forward(data)
                loss = loss_fn(output, target)

                test_loss += loss.item()*data.size(0)  # Add loss to running loss
                _, predicted = torch.max(output, dim=1)  # get predicted class
                test_correct += (predicted == target).sum().item()  # update total correct

                total_samples += data.size(0)  # update total samples

        testing_loss.append(test_loss / total_samples)   # Append average test loss to list
        test_accuracy.append(test_correct / total_samples)   # Append average test accuracy to list
    
    # Return loss and accuracy values for both training and testing
    return training_loss, testing_loss, training_accuracy, test_accuracy

In [ ]:
if __name__ == "__main__":
    """The dataset loaders were provided for you.
    You need to implement your own training process.
    You need plot the loss and accuracies during the training process and test process. 
    """
    
    indim = 10
    outdim = 2
    hidden_dim = 100
    lr = 0.01
    batch_size = 64
    epochs = 200

    #dataset
    Xtrain = np.loadtxt("/content/XTrain.txt", delimiter="\t")
    Ytrain = np.loadtxt("/content/yTrain.txt", delimiter="\t").astype(int)
    m1, n1 = Xtrain.shape
    print(m1, n1)
    train_ds = DS(Xtrain, Ytrain)
    train_loader = DataLoader(train_ds, batch_size=batch_size)

    Xtest = np.loadtxt("/content/XTest.txt", delimiter="\t")
    Ytest = np.loadtxt("/content/yTest.txt", delimiter="\t").astype(int)
    m2, n2 = Xtest.shape
    print(m1, n2)
    test_ds = DS(Xtest, Ytest)
    test_loader = DataLoader(test_ds, batch_size=batch_size)

    #construct the model
    model = SingleLayerMLP(indim, outdim, hidden_dim)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr = 0.01)

    #construct the training and testing process
    training_loss, testing_loss, training_acc, testing_acc = train_test(train_loader, test_loader, model, epochs, optimizer, criterion)

500 10
500 10


### Plot Training and Test losses

In [ ]:
fig = go.Figure()

# Add markers at the minimum values of the training and test losses
train_min_index = training_loss.index(min(training_loss))
test_min_index = testing_loss.index(min(testing_loss))

# Add training and test loss traces to the figure
fig.add_trace(go.Scatter(x=list(range(len(training_loss))), y=training_loss, name='Training Loss', line=dict(color='green')))
fig.add_annotation(x=train_min_index, y=training_loss[train_min_index], text=f"Min Training Loss: {round(training_loss[train_min_index], 2)}", showarrow=True, arrowhead=1, ax=-50, ay=-50)

fig.add_trace(go.Scatter(x=list(range(len(testing_loss))), y=testing_loss, name='Test Loss', line=dict(color='red')))
fig.add_annotation(x=test_min_index, y=testing_loss[test_min_index], text=f"Min Test Loss: {round(testing_loss[test_min_index], 2)}", showarrow=True, arrowhead=1, ax=-50, ay=50)

fig.update_layout(title='Training vs. Test Losses for Reference NN Implementation', xaxis_title='Epochs', yaxis_title='Loss', height=600, width= 800)
fig.show()

### Plot Training and Test Accuracy

In [ ]:
fig = go.Figure()

# Add training and test loss traces to the figure
fig.add_trace(go.Scatter(x=list(range(len(training_acc))), y=training_acc, name='Training Loss', line=dict(color='green')))
fig.add_trace(go.Scatter(x=list(range(len(testing_acc))), y=testing_acc, name='Test Loss', line=dict(color='red')))

# Add markers at the minimum values of the training and test losses
train_min_index = len(training_acc)-1
test_min_index = len(testing_acc)-1

fig.add_annotation(x=len(training_acc)-1, y=training_acc[-1], text=f"Max Training Accuracy: {round(training_acc[-1], 2)}", showarrow=True, arrowhead=1, ax=-50, ay=-50)
fig.add_annotation(x=test_min_index, y=testing_acc[-1], text=f"Max Test Accuracy: {round(testing_acc[-1], 2)}", showarrow=True, arrowhead=1, ax=-50, ay=50)

fig.update_layout(title='Training vs. Test Accuracies for Scratch NN Implementation', xaxis_title='Epochs', yaxis_title='Loss', height=600, width= 800)
fig.show()